In [1]:
from retrievers import CustomMultiQueryRetriever, CustomContextualCompressionRetriever, VectorStoreRetriever
from composers import ChainOfThoughtComposer
from config import openai_key
from functions import devrev_functions
import logging
import json

### Let's setup logging, to catch any errors that might creep up

In [2]:
logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)

## Setup Retriever

In [8]:
retriever = CustomMultiQueryRetriever(
    openai_key=openai_key,
    name="cmqr_vs_0",
    init_functions=devrev_functions
)

## Setup Composer

In [9]:
composer = ChainOfThoughtComposer(
    openai_api_key=openai_key,
    temperature=0.0,
    max_tokens=1000,
)

## Run a Query

In [10]:
query = "If there are more than 10 high-priority tasks, prioritize them; else, get the current user's ID"
functions = retriever.find_functions(query)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. Count the number of high-priority tasks', '2. Check if the count from #1 is greater than 10', '3. If the count from #1 is greater than 10, prioritize the high-priority tasks', "4. If the count from #1 is not greater than 10, get the current user's ID"]


In [11]:
functions

[Document(page_content='{"name": "prioritize_objects", "description": "Returns a list of objects sorted by priority. The logic of what constitutes priority for a given object is an internal implementation detail.", "parameters": {"type": "object", "properties": {"objects": {"title": "objects", "type": "array of objects", "description": "A list of objects to be prioritized."}}, "required": []}}', metadata={'index': 2}),
 Document(page_content='{"name": "create_actionable_tasks_from_text", "description": "Given a text, extracts actionable insights, and creates tasks for them, which are kind of a work item.", "parameters": {"type": "object", "properties": {"text": {"title": "text", "type": "string", "description": "The text from which the actionable insights need to be created."}}, "required": []}}', metadata={'index': 7}),
 Document(page_content='{"name": "resolution_status", "description": "Gives an array of only unresolved issues", "parameters": {"type": "object", "properties": {"unres

In [12]:
#format the answer json
answer = composer(query=query, functions=functions)
answer_json = json.loads(answer['text'])
print(answer['text'])

[
    {
        "tool_name": "works_list",
        "arguments": [
            {
                "argument_name": "issue.priority",
                "argument_value": "high"
            }
        ]
    },
    {
        "tool_name": "count_objects",
        "arguments": [
            {
                "argument_name": "objects",
                "argument_value": "$$PREV[0]"
            }
        ]
    },
    {
        "tool_name": "if_else_condition",
        "arguments": [
            {
                "argument_name": "Function",
                "argument_value": "$$PREV[1] > 10"
            },
            {
                "argument_name": "If_true",
                "argument_value": "prioritize_objects"
            },
            {
                "argument_name": "If_false",
                "argument_value": "who_am_i"
            }
        ]
    }
]
